## Load data from google sheet 
https://docs.google.com/spreadsheets/d/1ZSLM2gc6GBWs1fvvNwzA0D7jusRWv2xPVqOVoXcYJ3o/edit#gid=732292548

In [88]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('./questioniso-0a60ce8c39c1.json', scope)

# authorize the clientsheet 
client = gspread.authorize(creds)

sheet = client.open_by_key('1ZSLM2gc6GBWs1fvvNwzA0D7jusRWv2xPVqOVoXcYJ3o')

# get the first sheet of the Spreadsheet
sheet_instance = sheet.worksheet('news')

data = sheet_instance.get_all_records()
print('data', data[:1])

data [{'id': 1, 'title': 'Dow Jones tăng hơn 300 điểm, chấm dứt 5 phiên giảm liên tiếp', 'content': 'Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3), khi nhà đầu tư đặt cược vào việc nguy cơ lan truyền sau vụ phá sản của các ngân hàng Silicon Valley và Signature đã được ngăn chặn.\n\n\n\nKết thúc phiên giao dịch ngày thứ Ba, chỉ số Dow Jones tăng 336.26 điểm (tương đương 1.06%) lên 32,155.40 điểm, chấm dứt chuỗi 5 phiên lao dốc liên tiếp. Chỉ số S&P 500 tiến 1.65% lên 3,919.29 điểm. Chỉ số Nasdaq Composite cộng 2.14% lên 11,428.15 điểm.\n\nSự nhiệt tình mua vào các cổ phiếu ngân hàng của nhà đầu tư đã giảm đi phần nào vào buổi chiều. Tuy nhiên, nhiều cổ phiếu vẫn ghi nhận đà tăng, đánh dấu bước đảo chiều sau 2 phiên giảm sâu khi nhà đầu tư ngày càng tin tưởng rằng những ngân hàng đó sẽ không chịu chung số phận như ngân hàng Silicon Valley và Signature. Các nhà quản lý cho biết vào ngày 12/3 rằng họ đã xây dựng một kế hoạch để can thiệp cho tất cả những người gửi tiền ở 2 ngân hàng này.\

In [89]:
df = pd.DataFrame()
title = []
content = []
date = []
stock_price = []
status = []

for values in data:
    title.append(values['title'])
    content.append(values['content'])
    date.append(values['date'])
    stock_price.append(values['stock_price'])
    status.append(values['status'])

df['title'] = title
df['content'] = content
df['date'] = date
df['stock_price'] = stock_price
df['status'] = status

df[0:5]

,title,content,date,stock_price,status
0,"Dow Jones tăng hơn 300 điểm, chấm dứt 5 phiên ...",Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3)...,,100.0,1.0
1,Vàng thế giới lùi nhẹ khi lợi suất trái phiếu ...,Giá vàng giảm vào ngày thứ Ba (14/3) khi lợi s...,,98.0,-0.5
2,Xét xử vụ DongABank: 'Lạnh gáy' với khoản vay ...,Sau khi bị cáo Phùng Ngọc Khánh (cựu Chủ tịch ...,,95.0,-1.0
3,Cựu tổng giám đốc Ngân hàng Đông Á Trần Phương...,Cùng các đồng phạm gây thiệt hại tổng số tiền ...,,93.0,-1.0
4,Nóng: Ngân hàng Nhà nước giảm lãi suất điều hành,Ngân hàng Nhà nước (NHNN) vừa ban hành các quy...,,94.0,1.0


In [90]:
import pandas as pd
from datetime import date, timedelta
import re
from underthesea import word_tokenize, sent_tokenize
from gensim import corpora, models

def clean_text(text):
    text = str(text).lower() # lowercase text
    return text

df = df[df['stock_price'] != '']

documents = df['content'].values.tolist()
print('documents:', documents[0:1])
print('len:', len(documents))

documents: ['Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3), khi nhà đầu tư đặt cược vào việc nguy cơ lan truyền sau vụ phá sản của các ngân hàng Silicon Valley và Signature đã được ngăn chặn.\n\n\n\nKết thúc phiên giao dịch ngày thứ Ba, chỉ số Dow Jones tăng 336.26 điểm (tương đương 1.06%) lên 32,155.40 điểm, chấm dứt chuỗi 5 phiên lao dốc liên tiếp. Chỉ số S&P 500 tiến 1.65% lên 3,919.29 điểm. Chỉ số Nasdaq Composite cộng 2.14% lên 11,428.15 điểm.\n\nSự nhiệt tình mua vào các cổ phiếu ngân hàng của nhà đầu tư đã giảm đi phần nào vào buổi chiều. Tuy nhiên, nhiều cổ phiếu vẫn ghi nhận đà tăng, đánh dấu bước đảo chiều sau 2 phiên giảm sâu khi nhà đầu tư ngày càng tin tưởng rằng những ngân hàng đó sẽ không chịu chung số phận như ngân hàng Silicon Valley và Signature. Các nhà quản lý cho biết vào ngày 12/3 rằng họ đã xây dựng một kế hoạch để can thiệp cho tất cả những người gửi tiền ở 2 ngân hàng này.\n\nChứng chỉ quỹ SPDR S&P Regional Banking ETF tiến 2%, phục hồi phần nào sau đà lao dốc

In [91]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

def cv_texts_to_tokens(texts):
    docs_tokened = []
    documents_cleaned = []
    for doc in texts:
        words = word_tokenize(str(doc).lower(), format="text").split(' ')
        words = [word.strip(".,?!") for word in words if word.isalpha()]
        docs_tokened.append(words)
        documents_cleaned.append(' '.join(words))
    return docs_tokened, documents_cleaned

docs_tokened, documents_cleaned = cv_texts_to_tokens(documents)

In [92]:
import gensim
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

w2v_model = gensim.models.KeyedVectors.load_word2vec_format('../model/wiki.vi.model.bin', binary=True)

In [93]:
print(docs_tokened)

[['mỹ', 'vào', 'ngày', 'thứ', 'ba', 'khi', 'vào', 'việc', 'sau', 'vụ', 'của', 'các', 'silicon', 'valley', 'và', 'signature', 'đã', 'được', 'phiên', 'ngày', 'thứ', 'ba', 'dow', 'jones', 'tăng', 'điểm', 'lên', 'điểm', 'chuỗi', 'phiên', 'lao', 'dốc', 's', 'p', 'tiến', 'lên', 'điểm', 'nasdaq', 'composite', 'cộng', 'lên', 'điểm', 'sự', 'mua', 'vào', 'các', 'của', 'đã', 'giảm', 'đi', 'vào', 'buổi', 'chiều', 'nhiều', 'vẫn', 'đà', 'tăng', 'bước', 'đảo', 'chiều', 'sau', 'phiên', 'giảm', 'sâu', 'khi', 'rằng', 'những', 'đó', 'sẽ', 'không', 'chịu', 'chung', 'như', 'silicon', 'valley', 'và', 'signature', 'các', 'nhà', 'cho', 'biết', 'vào', 'ngày', 'rằng', 'họ', 'đã', 'một', 'để', 'cho', 'những', 'người', 'gửi', 'tiền', 'ở', 'này', 'quỹ', 'spdr', 's', 'p', 'regional', 'banking', 'etf', 'tiến', 'sau', 'đà', 'lao', 'dốc', 'trong', 'phiên', 'trước', 'đó', 'first', 'republic', 'bank', 'gần', 'sau', 'khi', 'gần', 'vào', 'ngày', 'keycorp', 'vọt', 'gần', 'trong', 'đợt', 'nhẹ', 'sau', 'khi', 'trượt', 'dốc',

### Tính trung bình word2vec để tạo ra mảng training chỉ 2 chiều:
(dựa theo: https://github.com/PradipNichite/Youtube-Tutorials/blob/main/Yotutube_WordVectors.ipynb)

In [94]:

pdTrain = pd.DataFrame({'tokens': docs_tokened})

def sent_vec(sent):
    vector_size = w2v_model.vector_size
    wv_res = np.zeros(vector_size)
    ctr = 1
    for w in sent:
        if w in w2v_model:
            ctr += 1
            wv_res += w2v_model[w]
    wv_res = wv_res/ctr
    return wv_res
pdTrain['vec'] = pdTrain['tokens'].apply(sent_vec)
print(pdTrain.head())

                                              tokens  \
0  [mỹ, vào, ngày, thứ, ba, khi, vào, việc, sau, ...   
1  [giá, vàng, giảm, vào, ngày, thứ, ba, khi, mỹ,...   
2  [sau, khi, phùng, ngọc, khánh, cựu, hđqt, m, c...   
3  [cùng, các, gây, tiền, trên, tỉ, đồng, cựu, tr...   
4  [nhnn, vừa, các, giảm, từ, điểm, qua, còn, nhi...   

                                                 vec  
0  [0.4521556866843076, 0.36772873731374384, -0.9...  
1  [0.3843028273714601, 0.5310362149381412, -0.91...  
2  [0.09241913974457357, 0.3054196947547671, -0.8...  
3  [0.2170247682347419, 0.3187902290997966, -0.67...  
4  [0.2316580660791205, 0.7523304409403729, -0.98...  


In [100]:
X_train = pdTrain['vec'].to_list()

def convert_text_to_train(texts):
    docs_tokened, _ = cv_texts_to_tokens(texts)
    pdData = pd.DataFrame({'tokens': docs_tokened})
    pdData['vec'] = pdData['tokens'].apply(sent_vec)
    print(pdData.head())
    return pdData['vec'].to_list()


In [96]:
y_train = df['status'].values.tolist()
statusNp = np.asarray(df['status'].values.tolist())
statusNp = np.expand_dims(statusNp, axis = 1)
print(statusNp.shape, statusNp, y_train)

(19, 1) [[ 1. ]
 [-0.5]
 [-1. ]
 [-1. ]
 [ 1. ]
 [-1. ]
 [-1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [-1. ]
 [ 0.9]
 [-1. ]
 [-1. ]
 [-1. ]
 [ 0.5]
 [ 0.1]
 [ 0.4]] [1.0, -0.5, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 0.9, -1.0, -1.0, -1.0, 0.5, 0.1, 0.4]


In [97]:
from sklearn.svm import SVR
clf = SVR(kernel='linear')
clf.fit(X_train, y_train)


SVR(kernel='linear')

In [101]:
test_data = [
    """
    Chính sách tăng lãi suất của Chủ tịch Fed Jerome Powell nhằm ghìm lạm phát đang bị đe dọa sau vụ sụp đổ của Silicon Valley Bank.
    """
]

X_test = convert_text_to_train(test_data)
print('X:\n', len(X_test), len(X_test[0]))

result = clf.predict(X_test)
print('result:\n', result)

                                              tokens  \
0  [tăng, của, fed, jerome, powell, nhằm, ghìm, đ...   

                                                 vec  
0  [0.059424844570457935, -0.17645556177012622, -...  
X:
 1 400
result:
 [-2.19100919]


## Version dùng mean numpy

In [112]:
print(w2v_model['ăn'])
print('m' in w2v_model)

[-4.0891525e-01  2.6084104e+00 -5.3833354e-01 -5.9752214e-01
  1.8656381e+00  6.6219044e-01 -3.7117357e+00 -4.5574969e-01
 -1.0396051e-01 -1.9559094e+00  9.7429109e-01 -4.0667385e-01
 -1.0615259e+00 -2.1646779e+00  9.3465680e-01 -1.4586806e+00
  5.8061433e-01 -3.7961490e+00  3.0390272e+00 -2.1457682e+00
 -5.7892412e-01  1.7761465e+00 -9.1266108e-01 -7.8270189e-02
  1.8554451e+00  1.5198513e+00 -6.2319130e-01 -2.2093023e-01
 -1.8296558e+00 -2.0173174e-01  1.8952109e+00 -6.9549829e-01
  1.8698134e-01  1.3218739e+00  2.4016383e+00  6.6352658e-02
 -2.7717566e-01 -2.4175718e+00 -1.1903220e+00 -7.9305309e-01
  1.3936094e+00  2.0585716e+00  2.4315950e-01 -8.1384414e-01
 -1.9550422e+00  2.2965547e-01 -1.5387051e-01 -1.1004342e+00
 -2.4111086e-01  1.3018471e-01 -3.8814726e-01 -9.9034458e-02
  1.6107775e+00  6.1631495e-01  1.9548521e+00  6.5761530e-01
 -1.9960139e+00  1.9396085e+00 -1.9843059e+00  8.8569456e-01
  6.8909150e-01 -1.4907720e+00 -8.0561894e-01 -1.4252144e-01
  8.3218837e-01 -7.37897

In [120]:
import numpy as np

def vectorize(sentence):
    words = word_tokenize(str(sentence).lower(), format="text").split(' ')
    words = [word.strip(".,?!") for word in words if word.isalpha()]
    words_vecs = [w2v_model[word] for word in words if word in w2v_model]
    
    if len(words_vecs) == 0:
        return np.zeros(400)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train = np.array([vectorize(sentence) for sentence in documents])
print(X_train.shape)

(19, 400)


In [121]:
from sklearn.svm import SVR
clf_1 = SVR(kernel='linear')
clf_1.fit(X_train, y_train)


SVR(kernel='linear')

In [123]:
test_data = [
    """
    Công ty Lọc hóa dầu Bình Sơn (BSR) lên kế hoạch bảo dưỡng lớn cho nhà máy lọc dầu Dung Quất, đề ra mức lãi giảm gần 90%.
    """
]

X_test = np.array([vectorize(sentence) for sentence in test_data])
print(X_test.shape)
print('X:\n', X_test)

result = clf_1.predict(X_test)
print('result:\n', result)

(1, 400)
X:
 [[ 0.38850942  0.10464204 -0.9577096  -0.37256026  0.23389089  0.88185835
   0.10116863  0.13173072 -0.3344859   0.22937334  0.8288219   0.7126737
  -0.3320357  -0.48318952  0.00314491 -0.6878798   0.46772274 -0.14373691
  -0.12396431 -0.7381272   0.30108276  0.35294417 -0.33553392  0.10188492
  -1.0696872  -0.6097121  -0.92718166  0.4930355   0.81550753 -0.17446913
   0.10795683 -0.1640809  -0.06091186  0.5743248  -0.08611198 -0.53096783
   0.07730834  0.341963    0.40540507  0.13439354 -0.50310946  0.11671042
   1.154089    0.85130805  0.32956108  0.4763059  -0.98409754 -1.0457631
   0.34032178  0.03418423 -0.17263567  0.50373465  0.30538163  0.29047686
   0.11044743  0.8140377  -0.22052911  0.66510034  0.36747724 -0.725286
   0.5249629  -0.5162898   0.29559237  0.87318057 -1.5603259   0.24828897
  -0.6180903   0.6598458   0.7546648   0.6865031  -0.02314603 -0.2760694
  -1.0750731  -0.6624368  -1.938914   -0.87500334 -0.87977946  1.1592664
   0.9792852   0.4131683  -0.43

In [127]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
sentence = 'Chúng_tôi là những nghiên_cứu_viên .'  

input_ids = torch.tensor([tokenizer.encode(sentence)])

with torch.no_grad():
    features = phobert(input_ids)  # Models outputs are now tuples
    print('features', features)

## With TensorFlow 2.0+:
# from transformers import TFAutoModel
# phobert = TFAutoModel.from_pretrained("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base-v2 were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens

features BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 3.8362e-02,  7.0703e-01, -1.3202e-01,  ..., -9.7446e-02,
           2.5193e-01,  3.4828e-01],
         [ 2.1041e-01,  2.3984e-01,  9.1064e-03,  ..., -3.2405e-04,
          -1.7492e-01,  4.0126e-02],
         [ 2.3745e-01,  9.8417e-03, -1.6509e-01,  ..., -4.3378e-02,
          -7.5782e-02,  4.6838e-02],
         ...,
         [ 2.3041e-01,  3.7583e-01,  1.7602e-02,  ...,  7.8472e-02,
           1.8661e-01,  5.2052e-02],
         [-2.3020e-01,  5.0276e-01,  1.0913e-01,  ..., -7.3260e-02,
           1.4339e-01,  1.8320e-01],
         [ 1.8813e-01,  6.2870e-01, -2.4809e-01,  ..., -4.8115e-02,
           1.6404e-01,  4.7204e-01]]]), pooler_output=tensor([[-4.6626e-02, -2.6561e-02, -1.0378e-01, -3.0149e-01,  2.5160e-01,
          1.0361e-01, -1.8122e-01,  2.5892e-01,  1.0942e-01,  2.4178e-01,
         -3.4443e-01,  2.0829e-01, -5.8292e-03, -6.5296e-02,  1.1208e-01,
          7.5874e-02,  2.9334e-03,  2.2507e-0